In [4]:
import pandas as pd
import os
import warnings
from tqdm.notebook import tqdm
import re
import massbalancemachine as mbm
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point

from scripts.helpers import *
from scripts.glamos_preprocess import *

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

In [7]:
df_pm = pd.read_csv(path_PMB_GLAMOS_csv + 'CH_wgms_dataset_topo.csv')
df_pm.head()

,YEAR,POINT_ID,GLACIER,FROM_DATE,TO_DATE,POINT_LAT,POINT_LON,POINT_ELEVATION,POINT_BALANCE,PERIOD,RGIId,aspect,slope,dis_from_border,topo
0,2003,01,oberaar,2002-10-06,2003-10-11,46.538806,8.233237,2389.812633,-6174,annual,RGI60-11.01509,1.146752,0.237373,57.982758,2345.0
1,2003,02,oberaar,2002-10-06,2003-10-11,46.536611,8.225514,2499.825727,-5310,annual,RGI60-11.01509,1.297745,0.106283,205.000000,2458.0
2,2003,03,oberaar,2002-10-06,2003-10-11,46.532136,8.207734,2679.854419,-4320,annual,RGI60-11.01509,1.183308,0.150800,295.655212,2637.0
3,2014,02,oberaar,2013-09-28,2014-09-26,46.536575,8.225487,2443.825843,-3960,annual,RGI60-11.01509,1.297745,0.106283,205.000000,2458.0
4,2015,02,oberaar,2014-09-26,2015-09-26,46.536548,8.225526,2457.825873,-4527,annual,RGI60-11.01509,1.297745,0.106283,205.000000,2458.0
